# 载入包

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split,cross_val_score
plt.style.use('ggplot')
%matplotlib inline

# == 目录====


=========== 目录===============
1.平衡数据简介
2.样本平衡分区
3.第一组模型运算
    ——填补:需要哪个数据，在填补哪个数据
    ——筛选：筛选的结果用于确定x，其他模型则直接使用这些x
    ——测试1：滚动测试
    ——测试2：滚动测试
    ——测试3：滚动测试
4.第二组模型运算
    ——构建回归
    ——测试评估
5.整合模型
    ——模型1*模型2
    ——管理模型关系：模型间与业务间
6.综合评分应用

# 路径与数据

In [18]:
import os
os.chdir(r"./data")

In [19]:
residual=pd.read_excel("残耗.xlsx")
energy=pd.read_excel("能耗.xlsx")
energy.head(3)

,编号,energy_consump,output,unit_paper_consump,quality_control_group,order_class,machine_type,brand
0,GA#C0001,1.252763,3.001,0.000000,2,1,1,7
1,GA#C0002,3.912023,3.251,9.082621,1,3,1,7
2,GA#C0003,3.401197,46.751,6.575872,3,2,1,7


# 第一组模型
## 平衡数据简介

In [21]:
residual['污染标注'].value_counts()#normalize=True

0.0    8707
1.0     293
Name: 污染标注, dtype: int64

## 样本分区：根据y取值分组

In [22]:
residual_one=residual[residual['污染标注']==1]
residual_zero=residual[residual['污染标注']==0]

## 根据y==0分区抽样——训练、测试与滚动测试

In [23]:
xdatas=[]
for t in range(6):
    xdata,_=train_test_split(residual_zero.iloc[:,[*range(1,33)]],
                             train_size=0.05,
                             random_state=t)
    xdatas.append(xdata)
xdatas[2]

,污染标注,v2样品序列,v1合成剂,v3燃料类型,v2添加渠道,v1执勤时效1,助燃点,烟碱量,气体综合,消耗烟脂,...,过滤时效,噪声,温控,偏离位,标注,起点位,撤回点位,收紧度,通路,lHH
3350,0.0,1.0,2.0,3.0,2.0,1.0,77.520,52.0,138.4,NaN,...,1226.4,NaN,51.4,363.0,75.000,103.327,437.4,6.580,30.733,NaN
7006,0.0,1.0,1.0,2.0,4.0,1.0,62.667,488.0,762.4,NaN,...,1213.4,7.660,31.4,230.0,58.887,53.637,229.4,26.680,33.833,1034.4
7151,0.0,1.0,2.0,2.0,5.0,1.0,32.493,14.0,68.4,30.2,...,729.4,5.320,38.4,205.0,41.220,49.391,221.4,5.487,30.453,8072.4
5158,0.0,1.0,2.0,2.0,2.0,1.0,38.660,268.0,444.4,NaN,...,1342.4,NaN,8.4,95.0,62.633,62.633,108.4,18.673,32.687,NaN
5171,0.0,1.0,2.0,2.0,2.0,1.0,54.873,875.0,1547.4,NaN,...,1117.4,NaN,34.4,202.0,54.833,67.411,232.4,7.620,31.027,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,0.0,2.0,1.0,2.0,5.0,1.0,226.253,1769.0,2551.4,NaN,...,828.4,5.587,65.4,865.0,49.340,70.918,812.4,58.133,41.967,1754.4
2603,0.0,1.0,1.0,1.0,2.0,1.0,31.900,298.0,537.4,35.4,...,393.4,6.347,34.4,75.0,34.873,33.721,114.4,5.027,30.407,2249.4
6863,0.0,1.0,2.0,4.0,2.0,1.0,210.760,2763.0,4039.4,32.4,...,3687.4,6.460,15.4,1123.0,231.127,221.308,800.4,10.800,31.887,20992.4
2665,0.0,1.0,1.0,1.0,5.0,1.0,33.853,18.0,89.4,NaN,...,28.4,NaN,32.4,220.0,30.947,32.640,175.4,6.727,30.773,17770.4


## 合并y==0与y==1

In [24]:
resid_1=residual_one.append(xdatas[0],sort=False)
resid_2=residual_one.append(xdatas[1],sort=False)
resid_3=residual_one.append(xdatas[2],sort=False)
resid_4=residual_one.append(xdatas[3],sort=False)
resid_5=residual_one.append(xdatas[4],sort=False)
resid_6=residual_one.append(xdatas[5],sort=False)

## 构建模型：缺失值填补

In [25]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
#---------------------------------------------------------
from missingpy import MissForest
imput=MissForest(n_estimators=10,min_samples_leaf=40)

# -----填补文件123456-----
resid_miss=[]
for i in [resid_1,resid_2,resid_3,resid_4,resid_5,resid_6]:
    resid_miss1=imput.fit_transform(i.iloc[:,[*range(1,33)]])
    resid_miss.append(resid_miss1) # resid_miss[0],即数据集1

data1=pd.DataFrame(resid_miss[0],columns=resid_1.iloc[:,[*range(1,33)]].columns)
data1

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


,污染标注,v2样品序列,v1合成剂,v3燃料类型,v2添加渠道,v1执勤时效1,助燃点,烟碱量,气体综合,消耗烟脂,...,过滤时效,噪声,温控,偏离位,标注,起点位,撤回点位,收紧度,通路,lHH
0,1.0,1.0,2.0,4.0,2.0,2.0,307.107,395.000000,749.40000,32.900000,...,7359.4,7.507000,67.440095,772.0,528.967,420.370,748.4,5.653,30.407,2986.400000
1,1.0,1.0,2.0,3.0,4.0,1.0,50.147,149.000000,251.40000,31.920864,...,1160.4,10.427000,40.751435,360.0,188.327,107.221,319.4,5.673,30.500,156.400000
2,1.0,1.0,2.0,4.0,4.0,1.0,229.633,2805.000000,4210.40000,33.311839,...,6070.4,9.109642,33.400000,611.0,196.327,288.952,715.4,10.033,30.847,3066.400000
3,1.0,1.0,2.0,2.0,4.0,1.0,29.467,1117.118537,1575.48527,30.000000,...,1045.4,27.733000,31.400000,105.0,53.933,48.054,223.4,5.687,30.553,2970.400000
4,1.0,1.0,2.0,4.0,1.0,2.0,274.807,3077.000000,4385.40000,29.400000,...,6947.4,9.700000,55.400000,118.0,195.820,231.603,186.4,20.873,33.620,3344.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,0.0,1.0,1.0,3.0,5.0,1.0,49.740,238.000000,464.40000,33.432816,...,1784.4,11.227000,37.400000,215.0,57.267,96.797,191.4,45.260,38.973,30898.400000
724,0.0,1.0,1.0,2.0,4.0,1.0,55.973,834.000000,1424.40000,33.291733,...,729.4,14.636838,40.400000,115.0,83.427,71.853,122.4,45.260,38.973,26163.202805
725,0.0,1.0,2.0,4.0,3.0,2.0,88.520,3347.000000,4906.40000,31.700000,...,3250.4,7.920000,34.400000,225.0,86.773,61.447,219.4,5.727,30.593,55.400000
726,0.0,1.0,2.0,2.0,5.0,1.0,97.533,11.000000,135.40000,33.305835,...,467.4,14.636838,22.400000,403.0,41.080,117.969,378.4,11.047,31.160,32.400000


## 对数据集1进行特征选择，以推广到其他数据集

In [26]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

x1,y1=data1.iloc[:,[*range(1,32)]],data1['污染标注']
rfr=RandomForestClassifier()
selector=RFE(rfr,n_features_to_select=10).fit(x1,y1)
data_select=pd.concat([data1['污染标注'],x1[x1.columns[selector.get_support(indices=True)]]],axis=1)
data_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   污染标注    728 non-null    float64
 1   助燃点     728 non-null    float64
 2   吸阻过滤    728 non-null    float64
 3   圆周点位    728 non-null    float64
 4   烟碱HW    728 non-null    float64
 5   钠元素     728 non-null    float64
 6   助燃      728 non-null    float64
 7   噪声      728 non-null    float64
 8   偏离位     728 non-null    float64
 9   标注      728 non-null    float64
 10  撤回点位    728 non-null    float64
dtypes: float64(11)
memory usage: 62.7 KB


## 保存筛选后的变量

In [28]:
x_columns = [i for i in data_select.columns if i not in ['污染标注']]         
x_columns
# x_columns=data_select.columns[1:].tolist()

['助燃点', '吸阻过滤', '圆周点位', '烟碱HW', '钠元素', '助燃', '噪声', '偏离位', '标注', '撤回点位']

## 构建数据2，评估分类模型

In [29]:
data2=pd.DataFrame(resid_miss[1],columns=resid_2.iloc[:,[*range(1,33)]].columns)
x2,y2=data2[x_columns],data2['污染标注']
sgd2=SGDClassifier(loss='log',random_state=123).fit(x2,y2)
print(sgd2.score(x2,y2))

#-----------或构建更多组模型-------------
data3=pd.DataFrame(resid_miss[2],columns=resid_3.iloc[:,[*range(1,33)]].columns)
x3,y3=data3[x_columns],data3['污染标注']
sgd3=SGDClassifier(loss='log',random_state=123).fit(x3,y3)
print(sgd3.score(x3,y3))

0.8145604395604396
0.3543956043956044


## 保存模型

In [31]:
from joblib import dump, load
dump(sgd3, 'C:/Users/Administrator/Desktop/sgd3模型.joblib')

['C:/Users/Administrator/Desktop/sgd3模型.joblib']

## 加载保存好的模型

In [32]:
sgdP=load('C:/Users/Administrator/Desktop/sgd3模型.joblib')                #加载模型3

## 填补所有数据,并对全体数据预测

In [33]:
residualAll=imput.fit_transform(residual.iloc[:,[*range(1,33)]])      #填补所有数据
residualMiss=pd.DataFrame(residualAll,columns=residual.iloc[:,[*range(1,33)]].columns)

residual_predict=sgdP.predict(residualMiss[x_columns])                #对全体数据预测
residual['预测值']=pd.DataFrame(residual_predict)
data_residual=residual[['编号','预测值']]
data_residual

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


,编号,预测值
0,GA#C0001,1.0
1,GA#C0002,1.0
2,GA#C0003,1.0
3,GA#C0004,1.0
4,GA#C0005,1.0
...,...,...
8995,GG#T1022,1.0
8996,GG#T1023,0.0
8997,GG#T1024,1.0
8998,GG#T1025,1.0


# 第二组模型

In [34]:
from sklearn.linear_model import LinearRegression,SGDRegressor

energy_x,energy_y=energy.iloc[:,[*range(2,8)]],energy.iloc[:,1]
ols=LinearRegression().fit(energy_x,energy_y)
ols.score(energy_x,energy_y)
energy_predict=ols.predict(energy_x)

energy['预测值']=pd.DataFrame(energy_predict)
data_energy=energy[['编号','预测值']]

# 第三阶段整合

In [35]:
#y1*y2=综合值
data_residual['评分']=data_residual['预测值']*data_energy['预测值']
data_residual[['编号','评分']].sort_values(['评分'],ascending=False)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_5872/272220261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_residual['评分']=data_residual['预测值']*data_energy['预测值']


,编号,评分
1,GA#C0002,3.592448
3164,GA#C3296,3.486231
3233,GA#C3367,3.399946
5333,GA#C5563,3.366615
4100,GA#C4275,3.358863
...,...,...
2926,GA#C3045,-0.221059
2101,GA#C2184,-0.230199
2311,GA#C2400,-0.230199
3163,GA#C3295,-0.319216
